In [ ]:
# ===========================
# 01_Segmentation.ipynb
# ===========================

# --- Imports ---
import os
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

import torch
from segment_anything import sam_model_registry, SamPredictor
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

plt.rcParams["figure.figsize"] = (8,6)

# =============================
# Paths & Basic Configuration
# =============================
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/final_project_computer_vision" #Change this to the root of where the poject is
DATA_DIR    = f"{PROJECT_DIR}/data"
IMAGES_DIR  = f"{DATA_DIR}/images"   # folder with .jpg/.png
OUT_DIR     = f"{PROJECT_DIR}/outputs"
MASK_DIR    = f"{OUT_DIR}/masks"
os.makedirs(MASK_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# =============================
# Load Models (SAM + GroundingDINO)
# =============================
device = "cuda" if torch.cuda.is_available() else "cpu"

GDINO_REPO  = "IDEA-Research/grounding-dino-base"
gd_processor = AutoProcessor.from_pretrained(GDINO_REPO)
gd_model     = AutoModelForZeroShotObjectDetection.from_pretrained(GDINO_REPO).to(device).eval()

# Put your SAM ViT-H weights in this path:
SAM_WEIGHTS = f"{PROJECT_DIR}/sam_vit_h_4b8939.pth"
assert os.path.isfile(SAM_WEIGHTS), "SAM checkpoint not found at SAM_WEIGHTS."

sam = sam_model_registry["vit_h"](checkpoint=SAM_WEIGHTS).to(device)
predictor = SamPredictor(sam)

  Preparing metadata (setup.py) ... done


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
# =============================
# Utility Functions
# =============================
def ensure_mask_size(mask_u8: np.ndarray, H: int, W: int) -> np.ndarray:
    """Return uint8 {0,255} mask with exact (H,W) size."""
    if mask_u8.ndim == 3:
        mask_u8 = cv2.cvtColor(mask_u8, cv2.COLOR_BGR2GRAY)
    if mask_u8.dtype != np.uint8:
        mask_u8 = mask_u8.astype(np.uint8)
    if mask_u8.max() in (0, 1):
        mask_u8 = (mask_u8 * 255).astype(np.uint8)
    if mask_u8.shape[:2] != (H, W):
        mask_u8 = cv2.resize(mask_u8, (W, H), cv2.INTER_NEAREST)
    return mask_u8

def refine_morphology_conservative(
    mask_u8: np.ndarray,
    min_island_area: int = 300,
    max_hole_area:   int = 300,
    band_px:         int = 2,
    lock_seed:       bool = True
) -> np.ndarray:
    """
    Cleanup: remove tiny FG islands, fill tiny holes,
    smooth edges ONLY within a narrow band, optionally never shrink seed.
    """
    H, W = mask_u8.shape[:2]
    seed = (mask_u8 > 0).astype(np.uint8)

    fg = seed.copy()
    num, labels, stats, _ = cv2.connectedComponentsWithStats(fg, connectivity=8)
    for i in range(1, num):
        if stats[i, cv2.CC_STAT_AREA] < min_island_area:
            fg[labels == i] = 0

    inv = (fg == 0).astype(np.uint8)
    border = inv.copy()
    mask_ff = np.zeros((H+2, W+2), np.uint8)
    cv2.floodFill(border, mask_ff, (0, 0), 2)
    bg_ext = (border == 2)
    holes  = (~bg_ext) & (inv == 1)
    num_h, lab_h, stats_h, _ = cv2.connectedComponentsWithStats(holes.astype(np.uint8), 8)
    for i in range(1, num_h):
        if stats_h[i, cv2.CC_STAT_AREA] <= max_hole_area:
            fg[lab_h == i] = 1

    if band_px > 0:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*band_px+1, 2*band_px+1))
        closed = cv2.morphologyEx(fg, cv2.MORPH_CLOSE, k, iterations=1)
        band = cv2.dilate(fg, k, 1) ^ cv2.erode(fg, k, 1)
        fg_smooth = fg.copy()
        fg_smooth[band > 0] = closed[band > 0]
    else:
        fg_smooth = fg

    if lock_seed:
        fg_smooth = np.maximum(fg_smooth, seed)

    return (fg_smooth.astype(np.uint8) * 255)



In [58]:
# =============================
# GroundingDINO (HF) helpers
# =============================
def gd_text_to_boxes_hf(
    img_rgb: np.ndarray,
    text_list,
    box_threshold: float = 0.25,
    text_threshold: float = 0.25,
    iou_nms: float = 0.5
):
    """Run GroundingDINO (HF) and return NMS'ed XYXY boxes, scores, labels."""
    H, W = img_rgb.shape[:2]
    image = Image.fromarray(img_rgb)

    inputs = gd_processor(images=image, text=text_list, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = gd_model(**inputs)

    post = getattr(gd_processor, "post_process_grounded_object_detection", None)
    if callable(post):
        try:
            results = post(outputs, inputs["input_ids"], box_threshold, text_threshold, [(H, W)])
        except TypeError:
            results = post(
                outputs, inputs["input_ids"], box_threshold, text_threshold,
                torch.tensor([[H, W]], device=outputs.logits.device)
            )
    else:
        post2 = getattr(gd_processor, "post_process_object_detection", None)
        if not callable(post2):
            raise RuntimeError("GroundingDINO post-process not found.")
        results = post2(outputs, threshold=box_threshold, target_sizes=[(H, W)])

    res = results[0] if isinstance(results, (list, tuple)) else results
    boxes  = res["boxes"].detach().cpu().numpy()
    scores = res["scores"].detach().cpu().numpy()
    labels = None
    if "text_labels" in res:
        labels = [str(t).lower() for t in res["text_labels"]]
    elif "labels" in res and hasattr(gd_processor, "tokenizer"):
        ids = res["labels"].detach().cpu().numpy().tolist()
        labels = [gd_processor.tokenizer.decode([i]).lower() for i in ids]

    def _nms_xyxy(b, s, thr=0.5):
        if len(b) == 0: return []
        b = b.astype(np.float32); s = s.astype(np.float32)
        x1,y1,x2,y2 = b[:,0],b[:,1],b[:,2],b[:,3]
        area = (x2-x1+1)*(y2-y1+1)
        order = s.argsort()[::-1]
        keep=[]
        while order.size > 0:
            i = order[0]; keep.append(i)
            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])
            w = np.maximum(0, xx2-xx1+1); h = np.maximum(0, yy2-yy1+1)
            inter = w*h
            iou = inter/(area[i]+area[order[1:]]-inter + 1e-6)
            inds = np.where(iou <= thr)[0]
            order = order[inds+1]
        return keep

    keep = _nms_xyxy(boxes, scores, iou_nms)
    boxes  = boxes[keep].astype(int).tolist()
    scores = scores[keep].tolist()
    if labels is not None:
        labels = [labels[i] for i in keep]
    return boxes, scores, labels

def sam_mask_from_box(img_rgb: np.ndarray, box_xyxy):
    """Run SAM on a single XYXY box and return the best mask (0/255) and score."""
    predictor.set_image(img_rgb)
    ib = np.array(box_xyxy, dtype=np.int32)[None, :]
    masks, scores, _ = predictor.predict(
        point_coords=None, point_labels=None, box=ib, multimask_output=True
    )
    best = int(np.argmax(scores))
    return (masks[best].astype(np.uint8)) * 255, float(scores[best])

In [59]:
# =============================
# Part detection configuration
# =============================
KW = {
    "windows": [
        "car window", "side window", "rear window", "back window",
        "windshield", "windscreen", "quarter glass", "rear windshield", "rear screen", "car glass"
    ],
    "wheels": [
        "wheel", "car wheel", "tire", "tyre", "rim", "hubcap", "alloy wheel"
    ],
    "lights": [
        "headlight", "head lamp", "headlamp", "front light", "fog light",
        "daytime running light", "turn signal", "indicator", "blinker"
    ],
}

def detect_part_boxes_in_car_roi(
    img_rgb: np.ndarray, car_box, text_list,
    scale: float = 1.8, pad: int = 16, box_thr: float = 0.25, text_thr: float = 0.20
):
    """Upscale a padded car ROI, run GroundingDINO, then map boxes back to full image coords."""
    H, W = img_rgb.shape[:2]
    x0, y0, x1, y1 = car_box
    x0p, y0p = max(0, x0 - pad), max(0, y0 - pad)
    x1p, y1p = min(W - 1, x1 + pad), min(H - 1, y1 + pad)
    roi = img_rgb[y0p:y1p+1, x0p:x1p+1]

    roi_h, roi_w = roi.shape[:2]
    newW, newH = int(roi_w * scale), int(roi_h * scale)
    roi_big = cv2.resize(roi, (newW, newH), interpolation=cv2.INTER_LINEAR)

    boxes_big, scores, labels = gd_text_to_boxes_hf(
        roi_big, text_list, box_threshold=box_thr, text_threshold=text_thr, iou_nms=0.5
    )

    boxes_full = []
    for bx in boxes_big:
        bx0 = int(bx[0] / scale) + x0p
        by0 = int(bx[1] / scale) + y0p
        bx1 = int(bx[2] / scale) + x0p
        by1 = int(bx[3] / scale) + y0p
        boxes_full.append([bx0, by0, bx1, by1])
    return boxes_full, scores, labels

def _box_area(b):
    return max(0, (b[2]-b[0]+1)) * max(0, (b[3]-b[1]+1))

def _center_in(box, car_box):
    x0,y0,x1,y1 = car_box
    cx = 0.5*(box[0]+box[2]); cy = 0.5*(box[1]+box[3])
    return (x0 <= cx <= x1) and (y0 <= cy <= y1)

def _iou(a, b):
    ax0,ay0,ax1,ay1 = a; bx0,by0,bx1,by1 = b
    xx0,yy0 = max(ax0,bx0), max(ay0,by0)
    xx1,yy1 = min(ax1,bx1), min(ay1,by1)
    w,h = max(0, xx1-xx0+1), max(0, yy1-yy0+1)
    inter = w*h
    return inter / (_box_area(a) + _box_area(b) - inter + 1e-6)

def _aspect_ratio_ok(b, rmin=0.7, rmax=1.3):
    w = max(1, b[2]-b[0]+1); h = max(1, b[3]-b[1]+1)
    r = w / h
    return (rmin <= r <= rmax)

def filter_window_boxes(car_box, boxes, labels, scores,
                        min_iou=0.10, min_frac=0.003, max_frac=0.55):
    x0,y0,x1,y1 = car_box
    car_area = max(1, (x1-x0+1) * (y1-y0+1))
    kept = []
    for b, l, s in zip(boxes, labels if labels else [None]*len(boxes), scores):
        name = (l or "").lower()
        if all(k not in name for k in ["window","windshield","windscreen","glass"]):
            continue
        frac = _box_area(b) / car_area
        if not (min_frac <= frac <= max_frac): continue
        if _iou(car_box, b) < min_iou:         continue
        if not _center_in(b, car_box):         continue
        kept.append((b, s))
    kept.sort(key=lambda t: t[1], reverse=True)
    return [b for b,_ in kept]

def filter_wheel_boxes(car_box, boxes, labels, scores,
                       min_iou=0.05, min_frac=0.008, max_frac=0.22, bottom_frac=0.45):
    x0,y0,x1,y1 = car_box
    car_area = max(1, (x1-x0+1) * (y1-y0+1))
    y_bottom = y0 + (y1-y0) * (1.0 - bottom_frac)
    kept = []
    for b, l, s in zip(boxes, labels if labels else [None]*len(boxes), scores):
        name = (l or "").lower()
        if all(k not in name for k in ["wheel","tire","tyre","rim","hubcap"]):
            continue
        frac = _box_area(b) / car_area
        if not (min_frac <= frac <= max_frac): continue
        if _iou(car_box, b) < min_iou:         continue
        if not _center_in(b, car_box):         continue
        cy = 0.5*(b[1]+b[3])
        if cy < y_bottom:                      continue
        if not _aspect_ratio_ok(b, 0.7, 1.35): continue
        kept.append((b, s))
    kept.sort(key=lambda t: t[1], reverse=True)
    return [b for b,_ in kept]

def filter_light_boxes(car_box, boxes, labels, scores,
                       min_iou=0.05, min_frac=0.002, max_frac=0.08, side_margin=0.35, lower_half=True):
    x0,y0,x1,y1 = car_box
    car_w, car_h = (x1-x0+1), (y1-y0+1)
    left_band  = x0 + car_w * side_margin
    right_band = x1 - car_w * side_margin
    y_mid = y0 + car_h * 0.5
    kept = []
    for b, l, s in zip(boxes, labels if labels else [None]*len(boxes), scores):
        name = (l or "").lower()
        if all(k not in name for k in ["headlight","head lamp","headlamp","front light","fog light","indicator","turn signal","blinker","daytime running"]):
            continue
        frac = _box_area(b) / max(1, car_w*car_h)
        if not (min_frac <= frac <= max_frac): continue
        if _iou(car_box, b) < min_iou:         continue
        if not _center_in(b, car_box):         continue
        cx = 0.5*(b[0]+b[2]); cy = 0.5*(b[1]+b[3])
        if not (cx <= left_band or cx >= right_band):
            continue
        if lower_half and cy < y_mid:
            continue
        kept.append((b, s))
    kept.sort(key=lambda t: t[1], reverse=True)
    return [b for b,_ in kept]


In [60]:
# =============================
# Build body mask (car minus parts)
# =============================
def build_body_mask(
    img_rgb: np.ndarray,
    box_thr: float = 0.22, text_thr: float = 0.25,
    empty_frac_guard: float = 0.30,
    win_scale: float = 1.5, wheel_scale: float = 1.8, light_scale: float = 1.0,
    roi_pad: int = 16
):
    H, W = img_rgb.shape[:2]

    car_boxes, car_scores, _ = gd_text_to_boxes_hf(img_rgb, ["car"], box_threshold=box_thr, text_threshold=text_thr)
    if not car_boxes:
        return None, "No car detected."
    car_boxes.sort(key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)
    car_box = car_boxes[0]

    car_mask_raw, _ = sam_mask_from_box(img_rgb, car_box)
    car_area = int((car_mask_raw > 0).sum())

    win_boxes_full, win_scores, win_labels = detect_part_boxes_in_car_roi(
        img_rgb, car_box, KW["windows"], scale=win_scale, pad=roi_pad, box_thr=box_thr, text_thr=text_thr
    )
    win_boxes = filter_window_boxes(car_box, win_boxes_full, win_labels, win_scores)

    wheel_boxes_full, wheel_scores, wheel_labels = detect_part_boxes_in_car_roi(
        img_rgb, car_box, KW["wheels"], scale=wheel_scale, pad=roi_pad, box_thr=box_thr, text_thr=text_thr
    )
    wheel_boxes = filter_wheel_boxes(car_box, wheel_boxes_full, wheel_labels, wheel_scores)

    light_boxes_full, light_scores, light_labels = detect_part_boxes_in_car_roi(
        img_rgb, car_box, KW["lights"], scale=light_scale, pad=roi_pad, box_thr=box_thr, text_thr=text_thr
    )
    light_boxes = filter_light_boxes(car_box, light_boxes_full, light_labels, light_scores)

    non_body = np.zeros((H, W), np.uint8)
    for bx in (win_boxes + wheel_boxes + light_boxes):
        m, _ = sam_mask_from_box(img_rgb, bx)
        non_body = np.maximum(non_body, m)

    body_raw = ((car_mask_raw > 0) & ~(non_body > 0)).astype(np.uint8) * 255
    body_area = int((body_raw > 0).sum())
    if car_area > 0 and (body_area / car_area) < (1.0 - empty_frac_guard):
        body_raw = (car_mask_raw > 0).astype(np.uint8) * 255

    return body_raw, None

In [ ]:
# =============================
# Batch over folder and save
# =============================

def tight_roi_from_mask(H, W, mask_u8, pad=8):
    """Small box around the current mask to constrain GrabCut search."""
    ys, xs = np.where(mask_u8>0)
    if len(xs)==0: return (0,0,W-1,H-1)
    x0,x1 = xs.min(), xs.max(); y0,y1 = ys.min(), ys.max()
    x0 = max(0, x0-pad); y0 = max(0, y0-pad)
    x1 = min(W-1, x1+pad); y1 = min(H-1, y1+pad)
    return (x0,y0,x1,y1)


def refine_grabcut_guarded(H, W, img_rgb, seed_mask_u8,
                           iters=1,
                           roi_pad=6,
                           sure_fg_dist=6, sure_bg_dist=8,
                           band_px=3,
                           edge_gate_px=1,
                           max_change_frac=0.01,
                           add_frac_max=0.001,
                           min_p95_gain=0.08):
    """
    """
    import numpy as np, cv2

    x0,y0,x1,y1 = tight_roi_from_mask(H, W, seed_mask_u8, pad=roi_pad)
    img_roi  = img_rgb[y0:y1+1, x0:x1+1]
    seed_roi = (seed_mask_u8[y0:y1+1, x0:x1+1] > 0).astype(np.uint8)

    dist_fg = cv2.distanceTransform(seed_roi, cv2.DIST_L2, 3)
    dist_bg = cv2.distanceTransform((1-seed_roi).astype(np.uint8), cv2.DIST_L2, 3)
    sure_fg = (dist_fg >= float(sure_fg_dist)).astype(np.uint8)
    sure_bg = (dist_bg >= float(sure_bg_dist)).astype(np.uint8)

    gc_mask = np.full(seed_roi.shape, cv2.GC_PR_FGD, dtype=np.uint8)
    gc_mask[sure_bg==1] = cv2.GC_BGD
    gc_mask[sure_fg==1] = cv2.GC_FGD

    bgdModel = np.zeros((1,65), np.float64); fgdModel = np.zeros((1,65), np.float64)
    cv2.grabCut(cv2.cvtColor(img_roi, cv2.COLOR_RGB2BGR),
                gc_mask, None, bgdModel, fgdModel, max(1,iters), cv2.GC_INIT_WITH_MASK)
    pred_roi = ((gc_mask==cv2.GC_FGD) | (gc_mask==cv2.GC_PR_FGD)).astype(np.uint8)

    k  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*band_px+1, 2*band_px+1))
    er = cv2.erode(seed_roi, k, 1)
    di = cv2.dilate(seed_roi, k, 1)
    band = (di > 0) & (er == 0)

    gray  = cv2.cvtColor(img_roi, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    distE = cv2.distanceTransform((edges==0).astype(np.uint8), cv2.DIST_L2, 3)
    nearE = (distE <= float(edge_gate_px))

    change_zone = band & nearE

    refined = seed_roi.copy()
    cand = pred_roi.copy()
    cand = np.minimum(cand, seed_roi)
    refined[change_zone] = cand[change_zone]

    change_frac = float(np.mean(refined != seed_roi))
    added_out   = float(np.sum((refined==1) & (seed_roi==0))) / max(1, int(seed_roi.sum()))

    def _p95(mask01):
        k3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        boundary = (cv2.dilate(mask01, k3, 1) > 0) & (cv2.erode(mask01, k3, 1) == 0)
        if boundary.sum() < 5: return np.inf
        return float(np.percentile(distE[boundary], 95))

    p95_seed = _p95(seed_roi)
    p95_ref  = _p95(refined)

    if (p95_seed - p95_ref) < float(min_p95_gain) or change_frac > float(max_change_frac) or added_out > float(add_frac_max):
        refined = seed_roi

    out_full = np.zeros((H,W), np.uint8)
    out_full[y0:y1+1, x0:x1+1] = (refined*255).astype(np.uint8)
    return out_full

paths = sorted(glob.glob(f"{IMAGES_DIR}/*.jpg") + glob.glob(f"{IMAGES_DIR}/*.png"))
for p in tqdm(paths):
    bgr = cv2.imread(p)
    if bgr is None:
        print(f"Skipped unreadable file: {p}")
        continue
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    Hc, Wc = rgb.shape[:2]

    body_mask, err = build_body_mask(rgb)
    if err:
        print(os.path.basename(p), " -> ", err)
        continue

    body_raw   = ensure_mask_size(body_mask, Hc, Wc)
    body_morph = refine_morphology_conservative(
        body_raw, min_island_area=300, max_hole_area=400, band_px=2, lock_seed=False
    )
    body_gc = refine_grabcut_guarded(
    Hc, Wc, rgb, body_morph,
    iters=2, roi_pad=6,
    sure_fg_dist=6, sure_bg_dist=9,
    band_px=2, edge_gate_px=1,
    max_change_frac=0.01, add_frac_max=0.001,
    min_p95_gain=0.12
    )

    stem    = os.path.splitext(os.path.basename(p))[0]
    out_dir = os.path.join(OUT_DIR, stem)
    os.makedirs(out_dir, exist_ok=True)

    cv2.imwrite(f"{out_dir}/body_raw.png",   body_raw)
    cv2.imwrite(f"{out_dir}/body_gc.png",    body_gc)

print("✅ Done")

100%|██████████| 20/20 [02:55<00:00,  8.79s/it]

✅ Done
